In [ ]:
def anotate_stats(df):
    
    import pandas as pd
    import numpy as np
    try:
        from scipy.stats import f_oneway
    except ImportError:
        !pip install scipy
        from scipy.stats import f_oneway
    
    ill = df[df['status'] == '1']
    not_ill = df[df['status'] == '0']
    summary_ill = ill.describe()
    summary_not_ill = not_ill.describe()
    oldnames = summary_ill.index.values.tolist()
    park = dict(zip(oldnames,[item + " (pd)" for i, item in enumerate(oldnames.copy())]))
    nopark = dict(zip(oldnames,[item + " (npd)" for i, item in enumerate(oldnames.copy())]))
    junt = pd.concat([renamevars(summary_ill.transpose(),park),renamevars(summary_not_ill.transpose(),nopark)], axis=1)
    junt = junt[['count (pd)', 'count (npd)','mean (pd)', 'mean (npd)','std (pd)', 'std (npd)','min (pd)', 'min (npd)','25% (pd)', '25% (npd)','50% (pd)', '50% (npd)','75% (pd)', '75% (npd)','max (pd)', 'max (npd)']]
    s, pval = f_oneway(junt['std (pd)'],junt['std (npd)'])
    
    def outlier_counter(df):

        cols = df.columns.difference(['name','status'])
        df = df[cols] #remove non numerial cols
        Q1 = df.quantile(0.25)
        Q3 = df.quantile(0.75)
        IQR = Q3-Q1
        return ((df < (Q1-1.5*IQR))|(df > (Q3+1.5*IQR))).sum()
        
    print(f"The total number of observations in this DataFrame is: {len(df)}"+"\n")
    print("There are apparent differences for some statistics in some variables, as can be seen in the table below:"+"\n")
    print(junt) #print the df
    print("\n"+ f"Variances are comparable between groups, as the p-value resulting from the F-test for equality of variances is greater than the significance level ({round(pval,2)} >> 0.05)"+"\n")
    print("Regarding outliers, no outliers are detected using the min and max values, as that those values represent the highest and lowest values of all the observations in all the columns. However, outlier detection methods can be used. One common approximation sets thresholds 1.5 times the IQR above and below each Q3 and Q1 pair (respectively)."+"\n")
    print(f"The outlier count in the Parkinson's group for each column is: \n \n{outlier_counter(ill)}. \n \n In total: {sum(outlier_counter(ill))} outliers. \n \n")
    print(f"The outlier count in the non-Parkinson's group for each column is: \n \n{outlier_counter(not_ill)} \n \n In total: {sum(outlier_counter(not_ill))} outliers. \n \n")
    print(f"A larger outlier count is found in the Parkinson's group. It makes sense as Parkinson's observation count is {junt['count (pd)'].HNR} and Non-Parkinson's {junt['count (npd)'].HNR}. As observation counts are unbalanced between groups, it would not be wise to remove the outliers, especially for the Non-Parkinson's group. Because valuable information could be lost, all the outliers are kept.")
